# Fitting a line to a dataset

## Introduction
Suppose we are given a set of data pairs, $\{(x_i,y_i)\}_{i=1,\ldots,N}$. Let's assume that the $x_i$ are known, but that the $y_i$ have uncertainties. That is, the measured value $y_i$ is within some range of the "true".  We further assume (perhaps we have some model or hypothesis) that the $y$ and $x$ are linearly related, although we don't know the slope or intercept of that line.

Because each of the $y_i$ has some range of uncertainty, no line will perfectly fit the data, but we can find a best fit line $y(x) = ax + b$ by adjusting the parameters $a$ and $b$ to maximize the probability of our dataset.  In practice this means finding $a$, $b$ that minimize (see chapter 8 of Taylor for details)
\begin{equation*}
    \chi^2 = \sum_{i=1}^N \frac{\left[y_i - y(x_i)\right]^2}{\sigma^2}.
\end{equation*}
Even though this line will be the one that fits the data the best, it doesn't necessarily imply that the fit is any good.  To determine the goodness-of-fit, we look at $\chi^2$ and compare its value to what we would expect if our assumptions are correct: namely, that the $y$ and $x$ pairs are linearly related, but the $y$ have some fluctuations that obey a gaussian relation with standard deviation $\sigma$.

To show how this works, we'll use `Python` to construct datasets that violate these assumputions, and then we'll see what happens to the $\chi^2$ values.  

## Set up the workspace
As before, we'll load in `numpy` and `matplotlib.pyplot`.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

We also need a custom `Python` class to make the fake datasets.  `Shift-click` in the following cell to load the class `sampleDataSets`.

In [ ]:
# %load sample_datasets.py
################################################################################
# Edward Brown
# Michigan State University
#
# Generates three data sets for a linear relation, one of which fits the data
# nearly perfectly and is therefore too good to be true; the other has 
# "outliers" -- points 10 standard deviations from the relation.
#
################################################################################

from numpy import linspace,zeros
from numpy.random import standard_normal, random, randint

class sampleDataSets:
    """
    Sets up a linear relation, y = m*x + b.  Datasets can be generated 
    from this relation by adding gaussian fluctuations to each y.  The std. 
    deviation of the fluctuation are chosen from a uniform random distribution 
    between 0.3 and 0.7.  There are 3 choices for datasets.
        1.  Fits the data much better than would be indicated by the size of its
            quoted uncertainties. The real fluctations have a std. dev. that is 
            1/5 of the quoted one.
        2.  Uncertainties are drawn from a normal distribution with a standard
            deviation matching the size of the errorbars.  This should produce
            an ideal chi^2 distribution if many trials are conducted.
        3.  Identical to 2, but 20% of the datapoints are given 5 sigma 
            fluctuations.
    """
    
    _slope = 3.0
    _intercept = 1.0
    _sig_low = 0.3
    _sig_high = 0.7

    def __init__(self):
        """
        Sets the relation, the nominal size of the errorbars, and the actual 
        size of the errorbars.
        """
        a = self._sig_low
        b = self._sig_high
        self.quoted_error = (b-a)*random() + a
        self.fake_error = 0.2*self.quoted_error
    
    def make_dataset(self,x,use_fake=False,with_outliers=False):
        """
        Constructs a dataset from the given relation with errorbars drawn from 
        a normal distribution.
        
        Arguments
        ---------
            x   :=  [array-like] the values at which the relation should be 
                    evaluated
            use_fake    :=  if True, then reduce the standard deviation of the 
                            fluctuations by a factor of 10.  This dataset will 
                            have an anomalously low chi^2.
            with_outliers:= if True, then 20% of the points will have 10 sigma 
                            fluctuations.
                            
        Returns
        -------
            y   := an ndarray of length(x.size) containing the dataset
        """
        
        m = self._slope
        b = self._intercept
        sig = self.fake_error if use_fake else self.quoted_error
        y = m*x + b + sig*standard_normal(len(x))
        if with_outliers:
            n = int(0.2*x.size)
            sgn = np.where(random(n) < 0.5,-1.0,1.0)
            indcs = randint(0,x.size-1,size=(2))
            y[indcs] = m*x[indcs] + b + sgn*5.0*sig
        return y
    
    @property
    def quoted_error(self):
        """
        returns the quoted standard deviation
        """
        return self._amp
    
    @quoted_error.setter
    def quoted_error(self,value):
        self._amp = value
        
    @property
    def fake_error(self):
        """
        returns a fake error
        """
        return self._fake
    
    @fake_error.setter
    def fake_error(self,value):
        self._fake = value
    
    def unrealistic_dataset(self,x):
        """
        Returns a dataset with actual uncertainties much less than the quoted 
        errorbars.
        
        Arguments
        ---------
            x   :=  [array-like] the values at which the relation should be 
                    evaluated
                            
        Returns
        -------
            y   := an ndarray of length(x.size) containing the dataset        
        """
        
        return self.make_dataset(x,use_fake=True)

    def realistic_dataset(self,x):
        """
        Returns a dataset with uncertainties that agree with the quoted 
        errorbars.
        
        Arguments
        ---------
            x   :=  [array-like] the values at which the relation should be 
                    evaluated
                            
        Returns
        -------
            y   := an ndarray of length(x.size) containing the dataset        
        """

        return self.make_dataset(x,use_fake=False)

    def dataset_with_outliers(self,x):
        """
        Returns a dataset with 80% of the points drawn from the normal 
        distribution, and 20% of the points having 10-sigma fluctuations.

        Arguments
        ---------
            x   :=  [array-like] the values at which the relation should be 
                    evaluated
                            
        Returns
        -------
            y   := an ndarray of length(x.size) containing the dataset        
        """

        return self.make_dataset(x,use_fake=False,with_outliers=True)
        
    def true_relation(self,x):
        """
        Returns the true relation, y = m*x + b

        Arguments
        ---------
            x   :=  [array-like] the values at which the relation should be 
                    evaluated
                            
        Returns
        -------
            y   :=  an ndarray of length(x.size) containing the underlying 
                    relation.
        """
        
        m = self._slope
        b = self._intercept
        return m*x + b


## Some sample datasets
Now we are ready to make some trial datasets.  Each dataset will consist of 10 points, `Ndata = 10`.  The line
```python
d = sampleDataSets()
```
initializes our sample: it sets the slope, intercept, and standard deviation of our points.  We then set some trial $x$-values in the range $[0.0,3.0]$
```python
x = np.linspace(0.0,3.0,Ndata)
```
calculate the true linear realation
```python
t = d.true_relation(xfit)
```
and show the results.

In [ ]:
Ndata = 10
d = sampleDataSets()
x = np.linspace(0.0,3.0,Ndata)
t = d.true_relation(x)
plt.plot(x,t)
plt.xlabel('x')
plt.ylabel('t')
plt.title('The actual relation')

Now let's see how some datasets might look.  We'll pick some $x$-values in the interval $[0,3)$ at random,

    x = 3.0*random(Ndata)
    
and generate three datasets: one with a realistic distribution of $y$ values, given the uncertainties; one with an unrealistic distribution of $y$ values; and one with two "outlier" points.  You can rerun this cell several times to get a different realization of the data.

In [ ]:
r = d.realistic_dataset(x)
f = d.unrealistic_dataset(x)
o = d.dataset_with_outliers(x)

plt.figure(figsize=(10,3))

plt.subplot(1,3,1)
plt.title('A realistic dataset')
plt.xlim(-0.3,3.3)
plt.errorbar(x,r,yerr=d.quoted_error,fmt='r.')
plt.plot(x,t)
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(1,3,2)
plt.title('An unrealistic dataset')
plt.xlim(-0.3,3.3)
plt.errorbar(x,f,yerr=d.quoted_error,fmt='r.')
plt.plot(x,t)
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(1,3,3)
plt.title('A dataset with outliers')
plt.xlim(-0.3,3.3)
plt.errorbar(x,o,yerr=d.quoted_error,fmt='r.')
plt.plot(x,t)
plt.xlabel('x')
plt.ylabel('y')

**Exercise:** Explain why the middle dataset is unrealistic. **Double-click in this box and type your answer below.**


## Fitting data

The numpy routine `polyfit(x,s,1)` will attempt to fit a polynomial of degree 1 (that is, a linear function $y=ax+b$) to the dataset `s`.  It does this by finding the values of $a$ and $b$ that minimize $\chi^2$. Let's try it with our 3 datasets.  As before, the blue line is the true relation; the black dashed line is the one with a minimum $\chi^2$.

In [ ]:
pr = np.polyfit(x,r,1)
yr = pr[0]*x + pr[1]
plt.figure(figsize=(10,3))
plt.subplot(1,3,1)
plt.errorbar(x,r,yerr=d.quoted_error,fmt='r.')
plt.plot(x,yr,color='black',linestyle='dashed')
plt.plot(x,t)
plt.xlabel('x')
plt.ylabel('y')
plt.title('A realistic dataset')

pf = np.polyfit(x,f,1)
yf = pf[0]*x + pf[1]
plt.subplot(1,3,2)
plt.errorbar(x,f,yerr=d.quoted_error,fmt='r.')
plt.plot(x,yf,color='black',linestyle='dashed')
plt.plot(x,t)
plt.xlabel('x')
plt.ylabel('y')
plt.title('An unrealistic dataset')

po = np.polyfit(x,o,1)
yo = po[0]*x + po[1]
plt.subplot(1,3,3)
plt.errorbar(x,o,yerr=d.quoted_error,fmt='r.')
plt.plot(x,yo,color='black',linestyle='dashed')
plt.plot(x,t)
plt.xlabel('x')
plt.ylabel('y')
plt.title('A dataset with outliers')

For these data sets, how does the fitted line compare with the actual relation?

**Type your response in this cell.**

## Testing the data
Now we'll look at the $\chi^2$ test for the data, where $\chi^2$ is defined in the introduction, above. First, we'll define a function `chi2`:

In [ ]:
def chi2(y,d,s):
    """
    Computes chi^2 for a set of fitted points and data.
    
    Arguments
    ---------
        y := [array] points y(x) from the fit
        d := [array] data points d(x)
        s := std.deviation
    
    Returns
    -------
        chi2
    """
    
    q2 = (y-d)**2 / s**2
    return np.sum(q2)

Now let's put it to the test.  We'll make `Ntrials = 1000` datasets.  We'll compute $\chi^2$ in two ways.  The first, using our "true" values for $a$ and $b$; the second, using an values of $a$ and $b$ that are best fit from the data.  

**Stop:** Before proceeding, *predict* how the values of $\chi^2$ computed with these two methods will differ. Do you think they will give the same values?  If not, which one will give a smaller value?

### The $\chi^2$ distribution
First, we'll allocate two arrays, each of length `Ntrials`, to hold our results.

```python
chi2fit = np.zeros(Ntrials)
chi2best = np.zeros(Ntrials)
```

Next, we'll make an array `t` of the "true" values of `y` for each `x`.

```python
t = d.true_relation(x)
```

Now we will run our trials.  For each trial, we'll make a realstic dataset, `s`, and then fit a line to that data and compute our estimated values of `y` for each `x`.

```python
p = np.polyfit(x,s,1)
y = x*p[0]+p[1]
```

Finally, we'll compute the $\chi^2$ for our "true" relation and for the fitted $a,b$.

```python
chi2fit[n] = chi2(t,s,d.quoted_error)
chi2best[n] = chi2(y,s,d.quoted_error)
```

We'll then plot the histograms of the two different $\chi^2$ distributions.

In [ ]:
Ntrials = 1000
chi2fit = np.zeros(Ntrials)
chi2best = np.zeros(Ntrials)
# t contains the true y-values
t = d.true_relation(x)

for n in range(Ntrials):
    # for each trial make a realistic dataset
    s = d.realistic_dataset(x)
    # fit it, and get the y-values for that fit
    p = np.polyfit(x,s,1)
    y = x*p[0]+p[1]
    # Now compute the chi-square value comparing the data points (s) against 
    # the true relation t and the fitted relation y
    chi2fit[n] = chi2(t,s,d.quoted_error)
    chi2best[n] = chi2(y,s,d.quoted_error)
alabel = '{0:5.2f}'.format(np.average(chi2fit))
flabel = '{0:5.2f}'.format(np.average(chi2best))
plt.xlabel(r'$\chi^2$')
plt.ylabel(r'$N$')
plt.hist(chi2fit,bins=20,align='mid',histtype='step',label=r'actual: $\langle\chi^2\rangle = '+alabel+r'$')
plt.hist(chi2best,bins=20,align='mid',histtype='step',label=r'best-fit: $\langle\chi^2\rangle = '+flabel+r'$')
plt.legend(frameon=False)

**Exercise:** Explain the difference between these two distributions of $\chi^2$. Are the averages what you expect? **Double-click in this box and type your answer below.**

**Exercise:** It is common to define a *reduced chi-square*, $\tilde{\chi}^2 = \chi^2/d$, where $d = N - c$ is the *degree of freedom* with $N$ the number of data points and $c$ the number of adjustable parameters in the fit.  See section 12.3 of Taylor for further discussion.  For the plot above, what is $\tilde{\chi}^2$ for the best-fit case? Explain why $\tilde{\chi}^2$ is a useful quantity.

### Testing our datasets
Now that we have seen the distribution of $\chi^2$ for our "realistic" dataset, let's look at how we can use this test to assess whether our fit is plausible.

Recall that we stored our datasets in variables `r` (realistic), `f` (fake), and `o` (with outliers).  We'll now fit each of those in turn, and compute a $\chi^2$ for each, which we'll store in the variables `chi2r`, `chi2f`, `chi2o`.

In [ ]:
p = np.polyfit(x,r,1)
y = x*p[0]+p[1]
chi2r = chi2(y,r,d.quoted_error)
print('chi-squared for realistic data = {0:5.2f}'.format(chi2r))

p = np.polyfit(x,f,1)
y = x*p[0]+p[1]
chi2f = chi2(y,f,d.quoted_error)
print('chi-squared for data with over-large error bars = {0:5.2f}'.format(chi2f))

p = np.polyfit(x,o,1)
y = x*p[0]+p[1]
chi2o = chi2(y,o,d.quoted_error)
print('chi-squared for data with outliers = {0:5.2f}'.format(chi2o))

Let's look at where these values of $\chi^2$ lie on our distribution `chi2best`.

In [ ]:
plt.title(r'$\chi^2$ for a dataset that matches hypothesis')
plt.xlabel(r'$\chi^2$')
plt.ylabel(r'$N$')
hst,bn,ptch = plt.hist(chi2best,bins=20,align='mid',histtype='step', label='dist. best-fit')
plt.vlines(chi2r, 0.0, max(hst), colors='k', linestyles='solid',label='matches hypothesis')
plt.legend(frameon=False)

**Exercise:** For this case, what is the probability that a dataset with random uncertainties would have a worse $\chi^2$?  Is it closest to 0, 0.5, or 1?

In [ ]:
plt.title(r'$\chi^2$ for a dataset with unrealistic errorbars')
plt.xlabel(r'$\chi^2$')
plt.ylabel(r'$N$')
hst,bn,ptch = plt.hist(chi2best,bins=20,align='mid',histtype='step', label='best-fit dist.')
plt.vlines(chi2f,0.0,max(hst),colors = 'g', linestyles='dashed', label='too large errorbars')
plt.legend(frameon=False)

**Exercise:** For this case, what is the probability that a dataset with random uncertainties would have a worse $\chi^2$?  Is it closest to 0, 0.5, or 1?

In [ ]:
plt.title(r'$\chi^2$ for a dataset with outliers')
plt.xlabel(r'$\chi^2$')
plt.ylabel(r'$N$')
hst,bn,ptch = plt.hist(chi2best,bins=20,align='mid',histtype='step', label='best-fit dist.')
plt.vlines(chi2o,0.0,max(hst),colors = 'r', linestyles='dotted', label='with outliers')
plt.legend(frameon=False)

**Exercise:** For this case, what is the probability that a dataset with random uncertainties would have a worse $\chi^2$?  Is it closest to 0, 0.5, or 1?

## The $\chi^2$ test
There is a theoretical distribution of $\chi^2$ values.  To see whether our fit is reasonable, we compute its value of $\chi^2$ and compare that value against the distribution.  Our question is, "If the dataset is as we expect, what is the probability of getting a value of $\chi^2$ larger than ours?"  If this is very unlikely, we conclude that our fit does not describe the data.

Alternatively, we may get a probability that is very close to 1; in other words, almost *any* dataset would have a worse fit.  This may suggest that the error bars are overestimated; alternatively, they could indicated the data has been faked or "fudged" to line up with the hypothesis.

The following block of code loads the `scipy.stats` module; it then defines a $\chi^2$ distribution with 8 degrees of freedom (10 data points - 2 adjustable parameters), and calculates the cumulative probability of getting a value of $\chi^2$ *larger* than `chi2r`, `chi2f`, and `chi2o`.  Then to make things interesting, we scramble the order of our probabilities and print them.

Your task: decide which probability matches the different datasets.

In [ ]:
import scipy.stats as st
df = 8# there are 10 data points and 2 adjustable parameters, so we have 8 degrees of freedom.
c2 = st.chi2(df)
ps = (c2.sf(chi2r),c2.sf(chi2f),c2.sf(chi2o))
# scramble the probabilities
ps = np.random.permutation(ps)
for lab, p in zip('abc',ps):
    print('{0}. {1:9.2e}'.format(lab,p))

**Exercise**: Edit this cell.  Beside each letter write the value of $p$ and the dataset — realistic, overlarge errorbars, or outliers — to which it corresponds.  Explain your reasoning.

a. 

b. 

c. 